# Calculate reproduction rate (R0) for infectious diseases within a micro-environment
This model calculates the reproduction rate (R0) of infectious diseases (Covid19) within a micro-environment based upon dispersion of an aerosol within an enclosed space. It assumes the prime method of transmission is an aerosol and that the aerosol is well mixed across the space, i.e. there are no local concentrations which may impact on an individual's probability of being infected.

** This notebook batch runs all of the environments defined within the configuration file using a multiprocess. The process is defined within a separate file so that it will run under windows **

The modelling follows the approach set out in the paper:
Buonanno, G., Stabile, L., & Morawska, L. (2020). Estimation of airborne viral emission: Quanta emission rate of SARS-CoV-2 for infection risk assessment [Preprint]. Infectious Diseases (except HIV/AIDS). https://doi.org/10.1101/2020.04.12.20062828

In [12]:
import math
import plotly.graph_objects as go
import plotly.express as px
import simpy
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm, tnrange

# Import local libraries
from Simulation import Simulation

In [13]:
file_db = pd.read_excel('./Configuration/Environment database.xlsx', header=4, engine='openpyxl')
environments = file_db['environment']

In [14]:
import multiprocess as mp
from run_parallel_simulation import run_parallel_simulation
sim_results = {}

for _, environment_name in environments.items():

    results = []

    max_iter = 1000
    with mp.Pool(max(1, int(os.cpu_count()*2/3-1))) as pool:
        results = list(tqdm(pool.imap(run_parallel_simulation, iter([environment_name] * max_iter)), desc=environment_name, total=max_iter))

    df = pd.DataFrame(results, columns=["Infections", "Total visitors", "Attack rate"])

    infections_mean = df.mean()['Infections']
    infections_sd = df.std()['Infections']
    total_visitors_mean = df.mean()['Total visitors']
    total_visitors_sd = df.std()['Total visitors']
    attack_rate = infections_mean / total_visitors_mean

    sim_results[environment_name] = (infections_mean, infections_sd, total_visitors_mean, total_visitors_sd, attack_rate)


In [15]:
df_results = pd.DataFrame.from_dict(sim_results, orient='index', columns=['infections mean', 'infections sd','visitors mean', 'visitors sd', 'attack rate'])
df_results

,infections mean,infections sd,visitors mean,visitors sd,attack rate
Pharmacy-natural-No Lockdown,4.576,2.079555,180.0,0.0,0.025422
Pharmacy-mechanical-No Lockdown,0.921,0.909718,180.0,0.0,0.005117
Pharmacy-natural-Lockdown,2.295,1.469740,90.0,0.0,0.025500
Pharmacy-mechanical-Lockdown,0.462,0.667084,90.0,0.0,0.005133
Supermarket-natural-No Lockdown,2.911,1.698813,360.0,0.0,0.008086
Supermarket-mechanical-No Lockdown,1.098,1.017562,360.0,0.0,0.003050
Supermarket-natural-Lockdown,0.458,0.677270,60.0,0.0,0.007633
Supermarket-mechanical-Lockdown,0.198,0.439305,60.0,0.0,0.003300
Restaurant-natural-No Lockdown,43.516,5.485785,160.0,0.0,0.271975
Restaurant-mechanical-No Lockdown,1.926,1.353646,160.0,0.0,0.012037


In [16]:
fig = px.scatter(df_results, x="visitors mean", y="attack rate", hover_data=[df_results.index])
fig.update_layout(
    title="Infections vs attack rate",
    xaxis_title="Infections (mean)",
    yaxis_title="Attack rate")
fig.show()

In [17]:
df_results.to_excel('./output/environment reproduction rates.xlsx', 'reproduction rates')